# Mid-Bootcamp Project - Classification

In [1]:
# import relevant libraries

import pymysql
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import getpass 

In [2]:
password = getpass.getpass()

connection_string = 'mysql+pymysql://root:' + password + '@localhost/credit_card_classification'
engine = create_engine(connection_string)

In [3]:
# pull data from mysql database into pandas dataframe

query = '''SELECT *
           FROM credit_card_data
           ;'''

data = pd.read_sql_query(query, engine)
data

,Customer_Number,Offer_Accepted,Reward,Mailer_Type,Income_Level,No_Bank_Accounts_Open,Overdraft_Protection,Credit_Rating,No_Credit_Cards_Held,No_Homes_Owned,Household_Size,Own_Your_Home,Average_Balance,Q1_Balance,Q2_Balance,Q3_Balance
0,1,No,Air Miles,Letter,High,1,No,High,2,1,4,No,1160.75,1669,877,1095
1,2,No,Air Miles,Letter,Medium,1,No,Medium,2,2,5,Yes,147.25,39,106,78
2,3,No,Air Miles,Postcard,High,2,No,Medium,2,1,2,Yes,276.50,367,352,145
3,4,No,Air Miles,Letter,Medium,2,No,High,1,1,4,No,1219.00,1578,1760,1119
4,5,No,Air Miles,Letter,Medium,1,No,Medium,2,1,6,Yes,1211.00,2140,1357,982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17971,17996,No,Cash Back,Letter,High,1,No,Low,1,1,5,Yes,167.50,136,65,71
17972,17997,No,Cash Back,Letter,High,1,No,Low,3,1,3,Yes,850.50,984,940,943
17973,17998,No,Cash Back,Letter,High,1,No,Low,2,1,4,No,1087.25,918,767,1170
17974,17999,No,Cash Back,Letter,Medium,1,No,Medium,4,2,2,Yes,1022.25,626,983,865


## Stage 1 - Data cleaning & Exploratory data analysis

In [8]:
data = data.set_index('Customer_Number')

In [10]:
data.shape, data.dtypes


((17976, 15),
 Offer_Accepted            object
 Reward                    object
 Mailer_Type               object
 Income_Level              object
 No_Bank_Accounts_Open      int64
 Overdraft_Protection      object
 Credit_Rating             object
 No_Credit_Cards_Held       int64
 No_Homes_Owned             int64
 Household_Size             int64
 Own_Your_Home             object
 Average_Balance          float64
 Q1_Balance                 int64
 Q2_Balance                 int64
 Q3_Balance                 int64
 dtype: object)

In [9]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
No_Bank_Accounts_Open,17976.0,1.255730,0.472405,1.00,1.00,1.0,1.00,3.00
No_Credit_Cards_Held,17976.0,1.903427,0.797086,1.00,1.00,2.0,2.00,4.00
No_Homes_Owned,17976.0,1.203549,0.427446,1.00,1.00,1.0,1.00,3.00
Household_Size,17976.0,3.499110,1.114476,1.00,3.00,3.0,4.00,9.00
Average_Balance,17976.0,940.515562,350.297837,48.25,787.50,1007.0,1153.25,3366.25
Q1_Balance,17976.0,910.450656,620.077060,0.00,392.75,772.0,1521.00,3450.00
Q2_Balance,17976.0,999.392190,457.402268,0.00,663.00,1032.0,1342.00,3421.00
Q3_Balance,17976.0,1042.033600,553.452599,0.00,633.00,945.5,1463.00,3823.00


In [11]:
data.isna().sum()

Offer_Accepted           0
Reward                   0
Mailer_Type              0
Income_Level             0
No_Bank_Accounts_Open    0
Overdraft_Protection     0
Credit_Rating            0
No_Credit_Cards_Held     0
No_Homes_Owned           0
Household_Size           0
Own_Your_Home            0
Average_Balance          0
Q1_Balance               0
Q2_Balance               0
Q3_Balance               0
dtype: int64